##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker を使用した音声分野での転移学習

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/audio_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub で表示</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>
</td>
  <td><a href="https://tfhub.dev/google/yamnet/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF Hub モデルを見る</a></td>
</table>

この colab ノートブックでは、[TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) を使用して、カスタム音声分類モデルをトレーニングする方法について説明します。

Model Maker ライブラリは、転移学習によって、カスタムデータセットを使用した TensorFlow Lite モデルのトレーニングプロセスを簡素化します。TensorFlow Lite モデルと独自のカスタムデータセットを維持すると、必要なトレーニングデータと時間の量が減ります。

これは、[音声モデルをカスタマイズして、Android でデプロイする Codelab](https://codelabs.developers.google.com/codelabs/tflite-audio-classification-custom-model-android) の一部です。

カスタム Birds データセットを使用します。スマートフォンで使用できる TFLite モデル、ブラウザでの推論に使用できる TensorFlow.JS モデル、サービスに使用できる SavedModel バージョンをエクスポートします。


## 依存関係のインストール


In [ ]:
!sudo apt -y install libportaudio2
!pip install tflite-model-maker

## TensorFlow、Model Maker、およびその他のライブラリのインポート

必要な依存関係のうち、TensorFlow と Model Maker を使用します。その他は、音声の操作、再生、視覚化用です。

In [ ]:
import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import glob
import random

from IPython.display import Audio, Image
from scipy.io import wavfile

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

## Birds データセット

Birds データセットは、次の 5 種類の鳥の鳴き声の教育コレクションです。

- ムナジロモリミソサザイ
- イエスズメ
- イスカ
- クリガシラジアリドリ
- ミヤマオナガカマドドリ

元の音声は、世界中の鳥の鳴き声を共有する Web サイトである [Xeno-canto](https://www.xeno-canto.org/) から取得されました。

まず、データをダウンロードします。

In [ ]:
birds_dataset_folder = tf.keras.utils.get_file('birds_dataset.zip',
                                                'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/birds_dataset.zip',
                                                cache_dir='./',
                                                cache_subdir='dataset',
                                                extract=True)
                                                

## データの観察

音声はすでにトレーニングフォルダとテストフォルダに分割されています。分割されたフォルダには、1 つのフォルダに 1 つの種類の鳥の鳴き声が格納されています。名前には、`bird_code` が使用されています。

音声はすべてモノラルで、サンプリングレートは 16kHz です。

各ファイルの詳細については、`metadata.csv` ファイルをお読みください。ファイルの作成者、ライセンス、詳細情報が記載されています。このチュートリアルでは読む必要はありません。

In [ ]:
# @title [Run this] Util functions and data structures.

data_dir = './dataset/small_birds_dataset'

bird_code_to_name = {
  'wbwwre1': 'White-breasted Wood-Wren',
  'houspa': 'House Sparrow',
  'redcro': 'Red Crossbill',  
  'chcant2': 'Chestnut-crowned Antpitta',
  'azaspi1': "Azara's Spinetail",   
}

birds_images = {
  'wbwwre1': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/Henicorhina_leucosticta_%28Cucarachero_pechiblanco%29_-_Juvenil_%2814037225664%29.jpg/640px-Henicorhina_leucosticta_%28Cucarachero_pechiblanco%29_-_Juvenil_%2814037225664%29.jpg', # 	Alejandro Bayer Tamayo from Armenia, Colombia 
  'houspa': 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/House_Sparrow%2C_England_-_May_09.jpg/571px-House_Sparrow%2C_England_-_May_09.jpg', # 	Diliff
  'redcro': 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Red_Crossbills_%28Male%29.jpg/640px-Red_Crossbills_%28Male%29.jpg', #  Elaine R. Wilson, www.naturespicsonline.com
  'chcant2': 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Chestnut-crowned_antpitta_%2846933264335%29.jpg/640px-Chestnut-crowned_antpitta_%2846933264335%29.jpg', # 	Mike's Birds from Riverside, CA, US
  'azaspi1': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/Synallaxis_azarae_76608368.jpg/640px-Synallaxis_azarae_76608368.jpg', # https://www.inaturalist.org/photos/76608368
}

test_files = os.path.abspath(os.path.join(data_dir, 'test/*/*.wav'))

def get_random_audio_file():
  test_list = glob.glob(test_files)
  random_audio_path = random.choice(test_list)
  return random_audio_path


def show_bird_data(audio_path):
  sample_rate, audio_data = wavfile.read(audio_path, 'rb')

  bird_code = audio_path.split('/')[-2]
  print(f'Bird name: {bird_code_to_name[bird_code]}')
  print(f'Bird code: {bird_code}')
  display(Image(birds_images[bird_code]))

  plttitle = f'{bird_code_to_name[bird_code]} ({bird_code})'
  plt.title(plttitle)
  plt.plot(audio_data)
  display(Audio(audio_data, rate=sample_rate))

print('functions and data structures created')

### 一部の音声の再生

データの理解を深めるために、テストフォルダのランダム音声ファイルを聴いてみます。

注意: このノートブックの後半では、この音声に対して推論を実行し、テストを行います。

In [ ]:
random_audio = get_random_audio_file()
show_bird_data(random_audio)

## モデルのトレーニング

音声で Model Maker を使用するときには、モデル仕様から始める必要があります。これは基本モデルであり、新しいモデルは情報を抽出して、新しいクラスを学習します。また、サンプリングレート、チャネル数などのモデル仕様パラメータに準拠するようにデータセットを変換する方法にも影響します。

[YAMNet](https://tfhub.dev/google/yamnet/1) は AudioSet データセットでトレーニングされた音声イベント分類器であり、AudioSet オントロジーから音声イベントを予測します。

入力は、16kHz、1 チャネルであると想定されています。

自分でリサンプリングする必要はありません。Model Maker で実行されます。

- `frame_length`: 各トレーニングサンプルの長さを決定します。この場合は EXPECTED_WAVEFORM_LENGTH * 3s です。

- `frame_steps`: トレーニングサンプルがどの程度離れているのかを決定します。この場合、サンプルは、(i-1)th サンプルの後、EXPECTED_WAVEFORM_LENGTH * 6s に開始します。

これらの値を設定する理由は、現実的なデータセットにおける一部の制限事項を回避するためです。

たとえば、Birds データセットでは、鳥が常に鳴いているわけではありません。鳥は鳴き、休んで、もう一度鳴きます。間にはノイズがあります。長いフレームでは、鳴き声を取り込むことができますが、あまり長く設定すると、トレーニングのサンプル数が減ってしまいます。


In [ ]:
spec = audio_classifier.YamNetSpec(
    keep_yamnet_and_custom_heads=True,
    frame_step=3 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH,
    frame_length=6 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH)

## データの読み込み

Model Maker には、フォルダからデータを読み込み、モデル仕様に合った形式にするための API があります。

トレーニングとテストの分割はフォルダに基づいています。検証データセットは、トレーニング分割の 20% として作成されます。

注意: `cache=True` はトレーニングを後から高速化するために重要ですが、データを格納するために必要な RAM の量が多くなります。Birds データセットは 300 MB しかないので問題にはなりませんが、独自のデータを使用する場合には、注意が必要です。


In [ ]:
train_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'train'), cache=True)
train_data, validation_data = train_data.split(0.8)
test_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'test'), cache=True)

## モデルをトレーニングする

audio_classifier には、モデルを作成し、すでにトレーニングを開始している [`create`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/create) メソッドがあります。

さまざまなパラメータをカスタマイズできます。詳細については、ドキュメントをお読みください。

今回は初めてなので、既定の構成をすべて使用し、100 エポックでトレーニングします。

注意: 最初のエポックは、キャッシュの作成時点であるため、すべてのエポックの中で一番長いエポックです。その後、各エポックに約 1 秒かかります。

In [ ]:
batch_size = 128
epochs = 100

print('Training the model')
model = audio_classifier.create(
    train_data,
    spec,
    validation_data,
    batch_size=batch_size,
    epochs=epochs)

精度は高いように見えますが、テストデータに対して評価ステップを実行し、シードが設定されていないデータに対してもモデルの結果が良好であることを検証することが重要です。

In [ ]:
print('Evaluating the model')
model.evaluate(test_data)

## モデルの理解

分類器をトレーニングする際、[混同行列](https://en.wikipedia.org/wiki/Confusion_matrix)を見ると役に立ちます。混同行列では、分類器がテストデータどどのように実行しているかを詳しく知ることができます。

Model Maker では、すでに混同行列が作成されています。

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  axis_labels = test_labels
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

## モデルのテスト [任意]

テストデータセットからサンプル音声に対してモデルを試し、結果を確認できます。

まず、サービングモデルを取得します。

In [ ]:
serving_model = model.create_serving_model()

print(f'Model\'s input shape and type: {serving_model.inputs}')
print(f'Model\'s output shape and type: {serving_model.outputs}')

前に読み込んだランダム音声に戻ります。

In [ ]:
# if you want to try another file just uncoment the line below
random_audio = get_random_audio_file()
show_bird_data(random_audio)

作成されたモデルには、固定入力ウィンドウがあります。

特定の音声ファイルでは、想定されたサイズのデータのウィンドウに分割する必要があります。最後のウィンドウは、ゼロ埋めにする必要があります。

In [ ]:
sample_rate, audio_data = wavfile.read(random_audio, 'rb')

audio_data = np.array(audio_data) / tf.int16.max
input_size = serving_model.input_shape[1]

splitted_audio_data = tf.signal.frame(audio_data, input_size, input_size, pad_end=True, pad_value=0)

print(f'Test audio path: {random_audio}')
print(f'Original size of the audio data: {len(audio_data)}')
print(f'Number of windows for inference: {len(splitted_audio_data)}')

すべての分割された音声をループし、それぞれに対してモデルを適用します。

トレーニングしたモデルには 2 つの出力があります。元の YAMNet の出力と、トレーニングした出力です。実際の環境は鳥の鳴き声よりももっと複雑であるため、この点は重要です。YAMNet の出力を使用して、関連しない音声を除外することができます。たとえば、鳥の鳴き声のユースケースでは、YAMNet で鳥か動物かが分類されない場合、モデルの出力で関連しない分類が実行されている可能性を示していると考えられます。

次の出力はいずれも表示され、関係を理解しやすくなっています。モデルにおけるほとんどの誤りは、YAMNet の予測が分野 (例: 鳥) に関連しないときに発生します。

In [ ]:
print(random_audio)

results = []
print('Result of the window ith:  your model class -> score,  (spec class -> score)')
for i, data in enumerate(splitted_audio_data):
  yamnet_output, inference = serving_model(data)
  results.append(inference[0].numpy())
  result_index = tf.argmax(inference[0])
  spec_result_index = tf.argmax(yamnet_output[0])
  t = spec._yamnet_labels()[spec_result_index]
  result_str = f'Result of the window {i}: ' \
  f'\t{test_data.index_to_label[result_index]} -> {inference[0][result_index].numpy():.3f}, ' \
  f'\t({spec._yamnet_labels()[spec_result_index]} -> {yamnet_output[0][spec_result_index]:.3f})'
  print(result_str)


results_np = np.array(results)
mean_results = results_np.mean(axis=0)
result_index = mean_results.argmax()
print(f'Mean result: {test_data.index_to_label[result_index]} -> {mean_results[result_index]}')

## モデルのエクスポート

最後のステップでは、埋め込まれたデバイスまたはブラウザで使用されるモデルをエクスポートします。

`export` メソッドでは、両方の形式がエクスポートされます。

In [ ]:
models_path = './birds_models'
print(f'Exporing the TFLite model to {models_path}')

model.export(models_path, tflite_filename='my_birds_model.tflite')

Python 環境でサービスを提供したり、使用したりするための SavedModel バージョンをエクスポートすることもできます。

In [ ]:
model.export(models_path, export_format=[mm.ExportFormat.SAVED_MODEL, mm.ExportFormat.LABEL])

## 次のステップ

完了しました。

これで、[TFLite AudioClassifier Task API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/audio_classifier) を使用して、モバイルデバイスに新しいモデルをデプロイできます。

異なるクラスが設定された独自のデータでも同じプロセスを試すことができます。[音声分類のための Model Maker](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier) のドキュメントを参照してください。

エンドツーエンドリファレンスアプリでも学習できます。[Android](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/android/) 版、[iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/ios) 版